In [ ]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 795.1 kB/s eta 0:00:00


In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv = '/content/drive/MyDrive/ADSDB/exploitation/data.csv'
df = pd.read_csv(csv)
df.head()

,Year,Quarter,Provinces,Sex,activity_rate,inflation_rate,house_price_index_type,house_price_index
0,2022,2022QIV,National Total,Both sexes,58.52,8.3906,General,142.666
1,2022,2022QIV,National Total,Both sexes,58.52,8.3906,New dwelling,157.801
2,2022,2022QIV,National Total,Both sexes,58.52,8.3906,Second-hand dwelling,140.353
3,2022,2022QIV,National Total,Males,63.53,8.3906,General,142.666
4,2022,2022QIV,National Total,Males,63.53,8.3906,New dwelling,157.801


In [ ]:
# Conncect to Duckdb
conn = duckdb.connect("/content/drive/MyDrive/ADSDB/colabs-part2/Analytical Sandbox/complete.db")

In [ ]:
conn.execute("""
CREATE OR REPLACE TABLE house_price_indexes AS
SELECT Year, Quarter, Provinces, house_price_index_type, house_price_index
FROM df
WHERE Sex = 'Both sexes';
""")

In [ ]:
conn.execute("DESCRIBE house_price_indexes").df()

,column_name,column_type,null,key,default,extra
0,Year,BIGINT,YES,None,None,NaN
1,Quarter,VARCHAR,YES,None,None,NaN
2,Provinces,VARCHAR,YES,None,None,NaN
3,house_price_index_type,VARCHAR,YES,None,None,NaN
4,house_price_index,DOUBLE,YES,None,None,NaN


In [ ]:
cursor = conn.cursor()

# Execute the query to get a list of all tables
cursor.execute("SHOW TABLES")

# Fetch all the results
tables = cursor.fetchall()

# Print the list of tables
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
house_price_indexes


In [ ]:
conn.execute("""SELECT * FROM house_price_indexes""").df()

,Year,Quarter,Provinces,house_price_index_type,house_price_index
0,2022,2022QIV,National Total,General,142.666
1,2022,2022QIV,National Total,New dwelling,157.801
2,2022,2022QIV,National Total,Second-hand dwelling,140.353
3,2022,2022QIII,National Total,General,143.860
4,2022,2022QIII,National Total,New dwelling,154.931
...,...,...,...,...,...
9160,2008,2008QII,Rioja. La,New dwelling,170.907
9161,2008,2008QII,Rioja. La,Second-hand dwelling,171.829
9162,2008,2008QI,Rioja. La,General,174.361
9163,2008,2008QI,Rioja. La,New dwelling,170.313


In [ ]:
conn.close()